In [23]:
import duckdb

# DuckDBに接続（インメモリで動作）
conn = duckdb.connect()

# DuckDBに拡張機能をインストール
conn.execute("""
INSTALL httpfs;
INSTALL json;
INSTALL spatial;
""")
conn.execute(f"""
LOAD httpfs;
LOAD json;
LOAD spatial;
""")
conn.execute(f"""
SET s3_region='us-west-2';
""")

In [24]:
# 国のテーブルを作成（領海を含む）

admin_geojson_url = "https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_0_countries.geojson"
conn.execute(f"CREATE TABLE countries AS SELECT * FROM ST_READ('{admin_geojson_url}')")

In [25]:
# 地震のテーブルを作成

eq_geojson_url = "https://earthquake.usgs.gov/fdsnws/event/1/query.geojson?starttime=1900-01-01%2000%3A00%3A00&minmagnitude=6&orderby=time"
conn.execute(f"CREATE TABLE earthquakes AS SELECT * FROM ST_READ('{eq_geojson_url}')")

In [26]:
# NOTE: closeするとDBの内容が失われるのでcloseしてはいけない
# conn.close()

In [27]:
# NOTE: connはgeojsonを読み込んだものを使い回す必要がある
# conn = duckdb.connect()
summary_of_tables = ""

# SHOWによってテーブル一覧を取得
show_result = conn.execute("SHOW").fetchall()
tables = [row[2] for row in show_result]

for table in tables:
    summary_of_tables += f"Table: {table}\n"
    # DESCRIBE TABLEの結果を文字列に変換
    describe_result = conn.execute(f"DESCRIBE TABLE {table}").fetchall()
    for row in describe_result:
        field_name = row[0]
        field_type = row[1]
        summary_of_tables += f"  Field: {field_name}, {field_type}\n"
print(summary_of_tables)

Table: countries
  Field: featurecla, VARCHAR
  Field: scalerank, INTEGER
  Field: LABELRANK, INTEGER
  Field: SOVEREIGNT, VARCHAR
  Field: SOV_A3, VARCHAR
  Field: ADM0_DIF, INTEGER
  Field: LEVEL, INTEGER
  Field: TYPE, VARCHAR
  Field: TLC, VARCHAR
  Field: ADMIN, VARCHAR
  Field: ADM0_A3, VARCHAR
  Field: GEOU_DIF, INTEGER
  Field: GEOUNIT, VARCHAR
  Field: GU_A3, VARCHAR
  Field: SU_DIF, INTEGER
  Field: SUBUNIT, VARCHAR
  Field: SU_A3, VARCHAR
  Field: BRK_DIFF, INTEGER
  Field: NAME, VARCHAR
  Field: NAME_LONG, VARCHAR
  Field: BRK_A3, VARCHAR
  Field: BRK_NAME, VARCHAR
  Field: BRK_GROUP, VARCHAR
  Field: ABBREV, VARCHAR
  Field: POSTAL, VARCHAR
  Field: FORMAL_EN, VARCHAR
  Field: FORMAL_FR, VARCHAR
  Field: NAME_CIAWF, VARCHAR
  Field: NOTE_ADM0, VARCHAR
  Field: NOTE_BRK, VARCHAR
  Field: NAME_SORT, VARCHAR
  Field: NAME_ALT, VARCHAR
  Field: MAPCOLOR7, INTEGER
  Field: MAPCOLOR8, INTEGER
  Field: MAPCOLOR9, INTEGER
  Field: MAPCOLOR13, INTEGER
  Field: POP_EST, DOUBLE
  Fie

In [44]:
input_text = "国の広さあたりの地震の回数が多い国トップ10を教えて下さい"
print(input_text)

国の広さあたりの地震の回数が多い国トップ10を教えて下さい


In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

# プロンプトの準備
template = """You are an expert of PostgreSQL and PostGIS.
You output the best PostgreSQL query based on given table schema and input text.

You will always reply according to the following rules:
- Output valid PostgreSQL query.
- The query MUST be return name, value and geom columns. Use AS to rename columns.
- The query MUST use ST_AsGeoJSON function to output geom column.
- The query MUST be line delimited and surrounded by just three backquote to indicate that it is a code block.

Hints:
- A3 field of countries table means three-letter country codes defined in ISO 3166-1
- ST_Point treats the SRID as 4326 by default, so there is no need to explicitly set ST_SetSRID.

** Table Schema: **
{table_schema}

User Input:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

res = chain.invoke({"input": input_text, "table_schema": summary_of_tables})
result = res.content.strip()
print(result)

```sql
SELECT
  c.NAME AS name,
  COUNT(e.id) / ST_Area(c.geom) AS value,
  ST_AsGeoJSON(c.geom) AS geom
FROM
  countries AS c
  JOIN earthquakes AS e ON ST_Intersects(c.geom, e.geom)
GROUP BY
  c.NAME,
  c.geom
ORDER BY
  value DESC
LIMIT
  10;
```


In [46]:
import re
match = re.search(r"```[^\n]*\n(.*?)```", result, re.DOTALL)

if match:
    query = match.group(1).strip()
    print(query)
else:
    print("SQLが見つかりませんでした。")

SELECT
  c.NAME AS name,
  COUNT(e.id) / ST_Area(c.geom) AS value,
  ST_AsGeoJSON(c.geom) AS geom
FROM
  countries AS c
  JOIN earthquakes AS e ON ST_Intersects(c.geom, e.geom)
GROUP BY
  c.NAME,
  c.geom
ORDER BY
  value DESC
LIMIT
  10;


In [47]:
duckdb_result = conn.execute(query).fetchall()
duckdb_result

[('Taiwan',
  23.987808024994848,
  '{"type":"Polygon","coordinates":[[[121.777818,24.394274],[121.175632,22.790857],[120.74708,21.970571],[120.220083,22.814861],[120.106189,23.556263],[120.69468,24.538451],[121.495044,25.295459],[121.951244,24.997596],[121.777818,24.394274]]]}'),
 ('Vanuatu',
  22.17553234756614,
  '{"type":"MultiPolygon","coordinates":[[[[167.216801,-15.891846],[167.844877,-16.466333],[167.515181,-16.59785],[167.180008,-16.159995],[167.216801,-15.891846]]],[[[166.793158,-15.668811],[166.649859,-15.392704],[166.629137,-14.626497],[167.107712,-14.93392],[167.270028,-15.740021],[167.001207,-15.614602],[166.793158,-15.668811]]]]}'),
 ('Solomon Is.',
  11.31540688439896,
  '{"type":"MultiPolygon","coordinates":[[[[162.119025,-10.482719],[162.398646,-10.826367],[161.700032,-10.820011],[161.319797,-10.204751],[161.917383,-10.446701],[162.119025,-10.482719]]],[[[161.679982,-9.599982],[161.529397,-9.784312],[160.788253,-8.917543],[160.579997,-8.320009],[160.920028,-8.320009],

In [48]:
# duckdb_resultをGeoJSONに変換する

import json

def duckdb_result_to_geojson(duckdb_result):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for row in duckdb_result:
        try:
            name, value, geom_str = row
            geom = json.loads(geom_str)
            feature = {
                "type": "Feature",
                "geometry": geom,
                "properties": {
                    "name": name,
                    "value": value
                }
            }
            geojson["features"].append(feature)
        except (ValueError, TypeError, IndexError) as e:
            print(f"Error processing row {row}: {e}")
            return None  # Return None if any row fails
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None

    return json.dumps(geojson, indent=2)



geojson_output = duckdb_result_to_geojson(duckdb_result)

if geojson_output:
    print(geojson_output)
else:
    print("Failed to convert DuckDB result to GeoJSON.")

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              121.777818,
              24.394274
            ],
            [
              121.175632,
              22.790857
            ],
            [
              120.74708,
              21.970571
            ],
            [
              120.220083,
              22.814861
            ],
            [
              120.106189,
              23.556263
            ],
            [
              120.69468,
              24.538451
            ],
            [
              121.495044,
              25.295459
            ],
            [
              121.951244,
              24.997596
            ],
            [
              121.777818,
              24.394274
            ]
          ]
        ]
      },
      "properties": {
        "name": "Taiwan",
        "value": 23.987808024994848
      }
    }

In [49]:
import folium
import json

geojson_data = json.loads(geojson_output)

m = folium.Map(location=[0, 0], zoom_start=2)  # default map

# Add the GeoJSON data to the map
folium.GeoJson(geojson_data).add_to(m)

# Get bounds from GeoJSON data and fit the map to those bounds
# Extract bounds from geojson_data
bounds = folium.GeoJson(geojson_data).get_bounds()
# Ensure bounds are valid before fitting
if bounds:
    m.fit_bounds(bounds)

# Display the map
display(m)